In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, BigInteger, Text, VARCHAR # <--- ¡ESTAS!
from sqlalchemy.sql import text # <--- ¡ESTA!
import mysql.connector
import numpy as np

# 1. Cargar las credenciales y definir el motor de conexión
load_dotenv('2.env')

CSV_FILENAME = 'reggaeton_data_2010_2024.csv' # Esto se usará solo si es necesario, pero la Celda 2 lo anula.
DB_USER = os.getenv("MYSQL_USER")
DB_PASSWORD = os.getenv("MYSQL_PASSWORD")
DB_HOST = os.getenv("MYSQL_HOST")
# Usaremos 'musica_db' directamente
DB_DATABASE = "musica_db"

engine = None # Inicializamos la variable

try:
    # --- PASO 1: CONECTARSE AL SERVIDOR Y CREAR LA BASE DE DATOS ---
   
    # 1.1 Conexión sin especificar la base de datos (Database=None)
    conn_no_db = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = conn_no_db.cursor()
   
    # 1.2 Crear la base de datos si no existe
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_DATABASE}")
    cursor.close()
    conn_no_db.close()
   
    print(f"✅ Base de datos '{DB_DATABASE}' creada o verificada en el servidor.")
   
   
    # --- PASO 2: CREAR EL ENGINE FINAL CON LA BASE DE DATOS ESPECIFICADA ---
   
    mysql_url = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_DATABASE}"
    engine = create_engine(mysql_url)
   
    print(f"✅ Conexión establecida a la base de datos MySQL: {DB_DATABASE}")

    print("\n--- ¡LISTO PARA LA CARGA DE DATOS Y MODELADO! ---")

except ImportError:
    print("❌ ERROR: Asegúrate de que instalaste 'mysql-connector-python' o 'mysqlclient'.")
except Exception as e:
    print(f"❌ ERROR CRÍTICO DE CONEXIÓN: Verifica que tu servidor MySQL esté encendido y tus credenciales en '2.env' sean correctas. Detalle: {e}")
    engine = None


✅ Base de datos 'musica_db' creada o verificada en el servidor.
✅ Conexión establecida a la base de datos MySQL: musica_db

--- ¡LISTO PARA LA CARGA DE DATOS Y MODELADO! ---


In [3]:
# --- CELDA 2: CARGA, CONSOLIDACIÓN Y MAPEO DE GÉNEROS ---

# 1. Lista de archivos y sus géneros correspondientes (AJUSTA ESTA LISTA)
file_info = [
    {'name': 'reggaeton_data_2010_2024.csv', 'genre': 'Reggaeton'},
    {'name': 'latin_data_2010_2024.csv', 'genre': 'Pop Latino'},
    {'name': 'funk_data_2010_2024.csv', 'genre': 'Funk'},
    {'name': 'indie_data_2010_2024.csv', 'genre': 'Rock Indie'},
    {'name': 'data_jazz_final2.csv', 'genre': 'jazz'},
    
    # {'name': 'indie_rock_data.csv', 'genre': 'Indie Rock'},
]

all_dataframes = []
print("--- INICIANDO CARGA Y CONCATENACIÓN DE GÉNEROS ---")

for info in file_info:
    filename = info['name']
    genre = info['genre']
    try:
        df = pd.read_csv(filename)
        
        df['Genero'] = genre
       
        # Opcional pero Recomendado: Renombrar para consistencia
        if 'Año de lanzamiento' in df.columns:
            df.rename(columns={'Año de lanzamiento': 'Año_lanzamiento'}, inplace=True)

        all_dataframes.append(df)
        print(f"   -> Cargado {filename} ({genre}) con {len(df)} filas.")
       
    except FileNotFoundError:
        print(f"   ❌ ERROR: Archivo NO encontrado: {filename}. ¡Asegúrate de que el nombre es EXACTO!")
    except Exception as e:
        print(f"   ❌ Error al leer {filename}: {e}")

df_consolidado = pd.concat(all_dataframes, ignore_index=True)

# 2. Deduplicación Final (vital para análisis multi-género)
initial_rows = len(df_consolidado)
df_consolidado.drop_duplicates(subset=['ID_Spotify'], keep='first', inplace=True)
final_rows = len(df_consolidado)
print(f"\nTotal ÚNICO de canciones (Global): {final_rows}")

# --- PREPARACIÓN DE LA TABLA GENEROS Y MAPEO ---
# 3. Extraer géneros únicos
df_generos_map = df_consolidado[['Genero']].drop_duplicates().reset_index(drop=True)

# 4. Asignar un ID temporal para la inserción (IMPORTANTE: MySQL le asignará el ID final)
# Esto asegura que tengamos el mismo orden en el DataFrame y en la BD.
df_generos_map['genero_id'] = df_generos_map.index + 1

# 5. Fusionar (Merge) el ID del género de vuelta al DataFrame consolidado
df_consolidado = pd.merge(
    df_consolidado,
    df_generos_map[['Genero', 'genero_id']],
    on='Genero',
    how='left'
)

# Creamos el DataFrame final para insertar en la tabla GENEROS
df_generos_db = df_generos_map[['genero_id', 'Genero']].rename(columns={'Genero': 'nombre_genero'})


--- INICIANDO CARGA Y CONCATENACIÓN DE GÉNEROS ---
   -> Cargado reggaeton_data_2010_2024.csv (Reggaeton) con 500 filas.
   -> Cargado latin_data_2010_2024.csv (Pop Latino) con 500 filas.
   -> Cargado funk_data_2010_2024.csv (Funk) con 500 filas.
   -> Cargado indie_data_2010_2024.csv (Rock Indie) con 500 filas.
   -> Cargado data_jazz_final2.csv (jazz) con 500 filas.

Total ÚNICO de canciones (Global): 2468


In [5]:
# --- CELDA 3: INSERCIÓN DE DATOS CON NORMALIZACIÓN EN MYSQL (CORREGIDA: SOLO AÑADIDA POPULARIDAD) ---

# Asegúrate de que las librerías necesarias estén importadas al inicio del notebook.
# from sqlalchemy.types import Integer, BigInteger, Text, VARCHAR 
# from sqlalchemy.sql import text

if 'engine' not in locals() or engine is None:
    print("❌ ERROR: El motor de conexión a MySQL no se creó correctamente.")
elif 'df_consolidado' not in locals():
    print("❌ ERROR: El DataFrame consolidado no se encontró. ¡Ejecuta la Celda 2 (Consolidación) primero!")
else:
    print("--- INICIANDO INSERCIÓN DE DATOS CONSOLIDADOS EN MYSQL ---")

    # Limpieza previa de tablas para evitar problemas con to_sql/replace y claves
    with engine.connect() as connection:
        connection.execute(text("DROP TABLE IF EXISTS CANCIONES"))
        connection.execute(text("DROP TABLE IF EXISTS ALBUMES"))
        connection.execute(text("DROP TABLE IF EXISTS ARTISTAS"))
        connection.execute(text("DROP TABLE IF EXISTS GENEROS"))
        connection.commit()

    # 0. Crear e insertar la tabla GENEROS
    df_generos_db.to_sql(
        'GENEROS',
        engine,
        if_exists='fail',   # ya hemos hecho DROP antes
        index=False,
        dtype={
            'genero_id': Integer,
            'nombre_genero': VARCHAR(50)
        }
    )

    with engine.connect() as connection:
        connection.execute(
            text(
                "ALTER TABLE GENEROS "
                "MODIFY genero_id INT NOT NULL AUTO_INCREMENT, "
                "ADD PRIMARY KEY (genero_id)"
            )
        )
        connection.commit()

    print(f"✅ Insertados {len(df_generos_db)} géneros únicos en la tabla GENEROS.")

    # 1. Preparar e insertar la tabla ARTISTAS (con datos de Last.fm)
    df_artistas = df_consolidado[
        [
            'Artista',
            'Playcount_LastFM',
            'Listeners_LastFM',
            'Biografia_Resumen'
        ]
    ].drop_duplicates(subset=['Artista']).dropna(subset=['Artista'])

    df_artistas = df_artistas.replace({np.nan: None})

    df_artistas.to_sql(
        'ARTISTAS',
        engine,
        if_exists='fail',
        index=False,
        dtype={
            'Artista': VARCHAR(255),
            'Playcount_LastFM': BigInteger,
            'Listeners_LastFM': BigInteger,
            'Biografia_Resumen': Text
        }
    )

    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE ARTISTAS ADD PRIMARY KEY (Artista)"))
        connection.commit()

    print(f"✅ Insertados {len(df_artistas)} artistas únicos en la tabla ARTISTAS (con métricas Last.fm).")

    # 2. Preparar e insertar la tabla ALBUMES
    df_albumes = df_consolidado[
        [
            'ID_Album',
            'Nombre_Album',
            'Año_Lanzamiento_Album',
            'Artista'  # FK a ARTISTAS
        ]
    ].drop_duplicates(subset=['ID_Album']).dropna(subset=['ID_Album'])

    df_albumes = df_albumes.replace({np.nan: None})

    df_albumes.to_sql(
        'ALBUMES',
        engine,
        if_exists='fail',
        index=False,
        dtype={
            'ID_Album': VARCHAR(50),
            'Nombre_Album': VARCHAR(255),
            'Año_Lanzamiento_Album': Integer,
            'Artista': VARCHAR(255),
        },
        chunksize=100
    )

    with engine.connect() as connection:
        connection.execute(text("ALTER TABLE ALBUMES ADD PRIMARY KEY (ID_Album)"))
        connection.execute(
            text("ALTER TABLE ALBUMES "
                 "ADD CONSTRAINT fk_albumes_artista "
                 "FOREIGN KEY (Artista) REFERENCES ARTISTAS(Artista)")
        )
        connection.commit()

    print(f"✅ Insertados {len(df_albumes)} álbumes únicos en la tabla ALBUMES.")

    # 3. Preparar e insertar la tabla CANCIONES (añadimos solo Popularidad)
    df_canciones = df_consolidado[
        [
            'ID_Spotify',
            'Nombre',
            'Año_lanzamiento',
            'ID_Album',
            'Artista',
            'genero_id',
            'Popularidad'  # campo añadido
        ]
    ].drop_duplicates(subset=['ID_Spotify']).dropna(subset=['ID_Spotify'])

    df_canciones = df_canciones.replace({np.nan: None})

    df_canciones.to_sql(
        'CANCIONES',
        engine,
        if_exists='fail',
        index=False,
        dtype={
            'ID_Spotify': VARCHAR(50),
            'Nombre': VARCHAR(255),
            'Año_lanzamiento': Integer,
            'ID_Album': VARCHAR(50),
            'Artista': VARCHAR(255),
            'genero_id': Integer,
            'Popularidad': Integer
        },
        chunksize=100
    )

    with engine.connect() as connection:
        # 3.1 Definir la PK
        connection.execute(text("ALTER TABLE CANCIONES ADD PRIMARY KEY (ID_Spotify)"))

        # 3.2 Añadir claves foráneas (con nombres de constraint explícitos)
        connection.execute(
            text("ALTER TABLE CANCIONES "
                 "ADD CONSTRAINT fk_canciones_album "
                 "FOREIGN KEY (ID_Album) REFERENCES ALBUMES(ID_Album)")
        )
        connection.execute(
            text("ALTER TABLE CANCIONES "
                 "ADD CONSTRAINT fk_canciones_genero "
                 "FOREIGN KEY (genero_id) REFERENCES GENEROS(genero_id)")
        )
        connection.execute(
            text("ALTER TABLE CANCIONES "
                 "ADD CONSTRAINT fk_canciones_artista "
                 "FOREIGN KEY (Artista) REFERENCES ARTISTAS(Artista)")
        )

        connection.commit()

    print(f"✅ Insertadas {len(df_canciones)} canciones únicas en la tabla CANCIONES (con Popularidad).")

    print("\n--- FASE 2 (MODELADO) COMPLETADA. Datos normalizados insertados en MySQL. ---")

# --- CELDA DE VERIFICACIÓN (AÑADIMOS LA POPULARIDAD) ---
try:
    query = """
    SELECT
        g.nombre_genero,
        COUNT(c.ID_Spotify) AS Total_Canciones,
        ROUND(AVG(c.Popularidad), 2) AS Avg_Popularidad
    FROM
        CANCIONES c
    JOIN
        GENEROS g ON c.genero_id = g.genero_id
    GROUP BY
        g.nombre_genero;
    """

    df_verification = pd.read_sql(query, engine)

    if df_verification.empty:
        print("⚠️ Advertencia: La tabla 'CANCIONES' está vacía o no se encontró.")
    else:
        print("\n--- VERIFICACIÓN DE GÉNEROS Y POPULARIDAD EN LA BASE DE DATOS ---")
        print(df_verification)

except Exception as e:
    print(f"❌ ERROR al verificar la base de datos: {e}")


--- INICIANDO INSERCIÓN DE DATOS CONSOLIDADOS EN MYSQL ---
✅ Insertados 5 géneros únicos en la tabla GENEROS.


C:\Users\PC\AppData\Local\Temp\ipykernel_19460\1988075318.py:23: UserWarning: The provided table name 'GENEROS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_generos_db.to_sql(
C:\Users\PC\AppData\Local\Temp\ipykernel_19460\1988075318.py:58: UserWarning: The provided table name 'ARTISTAS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_artistas.to_sql(


✅ Insertados 1251 artistas únicos en la tabla ARTISTAS (con métricas Last.fm).


C:\Users\PC\AppData\Local\Temp\ipykernel_19460\1988075318.py:89: UserWarning: The provided table name 'ALBUMES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_albumes.to_sql(


✅ Insertados 1490 álbumes únicos en la tabla ALBUMES.


C:\Users\PC\AppData\Local\Temp\ipykernel_19460\1988075318.py:129: UserWarning: The provided table name 'CANCIONES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df_canciones.to_sql(


✅ Insertadas 2468 canciones únicas en la tabla CANCIONES (con Popularidad).

--- FASE 2 (MODELADO) COMPLETADA. Datos normalizados insertados en MySQL. ---

--- VERIFICACIÓN DE GÉNEROS Y POPULARIDAD EN LA BASE DE DATOS ---
  nombre_genero  Total_Canciones  Avg_Popularidad
0          jazz              496              NaN
1          Funk              499            18.33
2    Pop Latino              490            40.54
3    Rock Indie              483            38.14
4     Reggaeton              500            45.30
